### OpenAI Access

First things first, you'll need to set-up an account on [OpenAI](platform.openai.com). Once you've done that - follow [these resources](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key) to create an API key. Make sure you save your API key!

In [76]:
import os

# Set the OPENAI_API_KEY environment variable
os.environ["OPENAI_API_KEY"] = ""

### OpenAI API Library

We'll be leveraging [this](https://github.com/openai/openai-python) library to access OpenAI's model endpoints.

There are a number of models to choose from and you can find resources about them [here](https://platform.openai.com/docs/models) and their pricing [here](https://openai.com/pricing).

The first step is to install `openai`!

In [77]:
!pip install openai -q

Once we've installed it, we need to import it and set our API key!

In [78]:
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")

If you wanted to use `gpt-4`, you'd need an account that has closed beta access to the model endpoint.

You can check if your API Key has access using the following cell.

In [79]:
# check if acct. has gpt-4 access
"gpt-4" in [model["root"] for model in openai.Model.list()["data"]]

True

For the rest of the tutorial, we're going to assume you're using `gpt-3.5-turbo` as your model.

Let's make some helper functions for prompting our model and generating our prompts.

In [80]:
def prompt_model(prompt_list, model="gpt-3.5-turbo"):
  return openai.ChatCompletion.create(model=model, messages=prompt_list)

def create_prompt(role, prompt):
  return {"role" : role, "content" : prompt}

As you can see, our prompts have to be in a specific format - as set by OpenAI.

Here's an example:

```
{"role" : "system", "content" : "You are an expert in Python programming."}
{"role" : "user", "content" : "Please define a function that provides the Nth number of the fibonacci sequence."}
```

Let's see that in action! Remember that you can feed OpenAI's chat completion endpoint with a list of prompts!

In [81]:
list_of_prompts = [
    {"role" : "system", "content" : "You are an expert in Python programming."},
    {"role" : "user", "content" : "Please define a function that provides the Nth number of the fibonacci sequence."}
]

model_output = prompt_model(list_of_prompts)
print(model_output)

{
  "id": "chatcmpl-89ZgtWVaPN3OrtDiyALSoKacgYZpG",
  "object": "chat.completion",
  "created": 1697292839,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Sure! Here's a function that calculates the Nth number of the Fibonacci sequence:\n\n```python\ndef fibonacci(n):\n    if n <= 0:\n        raise ValueError(\"N must be a positive integer.\")\n    elif n == 1:\n        return 0\n    elif n == 2:\n        return 1\n    else:\n        fib_prev = 0\n        fib_curr = 1\n        for _ in range(3, n + 1):\n            fib_prev, fib_curr = fib_curr, fib_prev + fib_curr\n        return fib_curr\n```\n\nExample usage:\n\n```python\nn = 10\nfib_number = fibonacci(n)\nprint(f\"The {n}th number in the Fibonacci sequence is: {fib_number}\")\n```\n\nOutput:\n\n```\nThe 10th number in the Fibonacci sequence is: 34\n```\n\nThis function handles non-positive values of N by raising a `ValueError` exception. F

As you can see, we get a lot of information back from the endpoint.

We can see the number of tokens we used, why the output stopped, what the output is, and more!

Let's view the prompt a bit clearer using some display libraries.

In [82]:
from IPython.display import display, Markdown

markdown_output = model_output["choices"][0]["message"]["content"]

display(Markdown(markdown_output))

Sure! Here's a function that calculates the Nth number of the Fibonacci sequence:

```python
def fibonacci(n):
    if n <= 0:
        raise ValueError("N must be a positive integer.")
    elif n == 1:
        return 0
    elif n == 2:
        return 1
    else:
        fib_prev = 0
        fib_curr = 1
        for _ in range(3, n + 1):
            fib_prev, fib_curr = fib_curr, fib_prev + fib_curr
        return fib_curr
```

Example usage:

```python
n = 10
fib_number = fibonacci(n)
print(f"The {n}th number in the Fibonacci sequence is: {fib_number}")
```

Output:

```
The 10th number in the Fibonacci sequence is: 34
```

This function handles non-positive values of N by raising a `ValueError` exception. Feel free to modify the function to suit your needs!

### Generating Synthetic Data

Alright, now we can pull everything together and start creating our synthetic data!

**NOTE:** Using OpenAI's endpoints to create our dataset does mean that we cannot use our model for commercial use. This is meant to demonstrate the methods, and can be extended to any open-source LLM.

We're going to use this process to create 100 product/marketing email pairs.

We'll be doing this in 2-steps:

1. Create the 100 products and short descriptions.
2. Create marketing emails for each of those 100 product/descriptions.

Let's begin by creating the prompt for our products/descriptions!

In [83]:
datagen_prompts = [
    {"role" : "system", "content" : "You are a product innovator. You create new products that people crave."},
    {"role" : "user", "content" : "Please generate a list of 10 new products and extremely short descriptions."},
]

In [84]:
first_data_gen = prompt_model(datagen_prompts)
print(first_data_gen["choices"][0]["message"]["content"])

1. Glow Up Goggles: Stylish, high-tech goggles that enhance your vision in low light conditions and make your eyes shine with a vibrant glow.

2. ZenScent Candles: Aromatherapy candles infused with calming scents like lavender and chamomile, designed to create a soothing atmosphere and promote relaxation.

3. SmartMeal: A futuristic meal replacement shake that instantly adapts to your body's nutritional needs, providing all the essential nutrients and vitamins required for optimal health in one convenient serving.

4. InstaGrow Planters: Intelligent self-watering planters equipped with sensors that monitor soil moisture levels and deliver the perfect amount of water and nutrients to your plants, ensuring lush greenery without the hassle of daily maintenance.

5. SweatShield Apparel: High-performance clothing engineered with advanced sweat-wicking technology to keep you dry and fresh, making it perfect for intense workouts or humid climates.

6. EchoBed Sleep System: A revolutionary mat

Okay, now that we have a list of 100 items - let's parse them out into a Python list - also, we can keep track of our total token usage to estimate costs!

In [85]:
def retrieve_token_usage(open_ai_response):
  return sum([tokens for tokens in open_ai_response["usage"].values()])

In [86]:
f"We used {retrieve_token_usage(first_data_gen)} tokens"

'We used 842 tokens'

The following code might need to be modified based on how your data was returned by OpenAI's endpoint!

In [88]:
text_response = first_data_gen["choices"][0]["message"]["content"]

products_and_descriptions = []
for line in text_response.splitlines():
  if "." in line:
    product_descriptions = line.split(".")[1]
    product_descriptions_split = product_descriptions.split(":")
    products_and_descriptions.append(
        {
            "product" : product_descriptions_split[0][1:],
            "description" : ":".join(product_descriptions_split[1:])[1:]+"."
        }
    )

In [89]:
products_and_descriptions[0]

{'product': 'Glow Up Goggles',
 'description': 'Stylish, high-tech goggles that enhance your vision in low light conditions and make your eyes shine with a vibrant glow.'}

Now that we have our items parsed out into a Python list - we can go ahead and iterate through each of the items and have whichever OpenAI model you selected create a short marketing email for it!

First though, we'll need a system prompt to use!

In [90]:
system_prompt = create_prompt(
    "system",
    "You are a marketing executive. You are proficient at writing short, and snappy marketing emails. The emails should be easy to read, and contain excited and vibrant language."
)

We'll also need a user prompt - we'll have to wrap this in a function so we can call it for each item of the 100 items we created above.

In [91]:
def generate_user_prompt(product, description):
  user_prompt = create_prompt(
      "user",
      f"Please create a short marketing email using this product: {product} and this description: {description}"
  )

  return user_prompt

Now we're good to start generating our synthetic data! We simply need to iterate through each item - and collate the results into a list of dictionaries!

(depending on which model you use, this step might take a long time, and could become expensive!)

In [92]:
from openai.error import RateLimitError
total_token_usage = 0

for idx, item in enumerate(products_and_descriptions):
  if "marketing_email" in item:
    continue
  print(f"Working on {idx}")
  user_prompt = generate_user_prompt(item["product"], item["description"])
  full_prompt = [system_prompt, user_prompt]
  try:
    prompt_response = prompt_model(full_prompt)
    item["marketing_email"] = prompt_response["choices"][0]["message"]["content"]
    total_token_usage += retrieve_token_usage(prompt_response)
  except RateLimitError as e:
    continue

Working on 0
Working on 1
Working on 2
Working on 3
Working on 4
Working on 5
Working on 6
Working on 7
Working on 8
Working on 9


In [93]:
products_desc_and_marktng_emails_dataset = [p_d_and_m for p_d_and_m in products_and_descriptions if "marketing_email" in p_d_and_m]

### Uploading Dataset to HuggingFace Hub

Now that we've created our synthetic dataset - let's push it to the HuggingFace hub!

As always, the first task is to get the required dependencies.

In [94]:
!pip install huggingface_hub -q

Now we can log-in to Hugging Face!

Make sure you have a Hugging Face account, and you have set up a read/write token!

More info here: https://huggingface.co/docs/hub/security-tokens

In [95]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

Now we can load our data into the desired format - and upload it to the hub!

In [96]:
!pip install datasets -q

In [97]:
from datasets import load_dataset, Dataset
import pandas as pd

In [98]:
hf_dataset = Dataset.from_pandas(pd.DataFrame(data=products_desc_and_marktng_emails_dataset))

In [99]:
hf_dataset

Dataset({
    features: ['product', 'description', 'marketing_email'],
    num_rows: 10
})

In [100]:
hf_username = "renatomoulin"
dataset_name = "fourthbrain_synthetic_marketmail_gpt35_turbo"

hf_dataset.push_to_hub(f"{hf_username}/{dataset_name}")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

### Conclusion

And that's it! You just created a synthetic dataset and pushed it to the hub!

Next stop? [Modeling!](https://colab.research.google.com/drive/1RfUuzG11Q8AaZuJIHLzXCVC087xoDeSd?usp=sharing)